# Welcome to glupyter-lite!

This notebook contains an example of glue-jupyter running inside of [JupyterLite](https://jupyterlite.readthedocs.io/en/stable/), which means that this is running entirely in the browser! This did require a few specific customizations that I'll describe below.

In [ ]:
import piplite

Our dependencies `fast-histogram` and `mpl-scatter-density` both require C extensions, so we need to build Pyodide-compatible wheels using Emscripten. There are a couple of other packages that didn't have compatible wheels available (note that Vispy still doesn't work yet, despite us building our own wheel, due to some runtime issues with emscripten not being recognized as an environment).

In [ ]:
await piplite.install('shapely')
await piplite.install('pypi/fast_histogram-0.14-cp39-abi3-pyodide_2024_0_wasm32.whl')
await piplite.install('pypi/mpl_scatter_density-0.8-py3-none-any.whl')
await piplite.install('pypi/glfw-2.9.0-py3-none-any.whl')
await piplite.install('pypi/vispy-0.15.2-cp312-cp312-pyodide_2024_0_wasm32.whl')

In [ ]:
# Do things like this to get around the fact that pyzmq is missing
await piplite.install("glue-core")
await piplite.install("bqplot")
await piplite.install("bqplot_image_gl")
await piplite.install("ipykernel")
await piplite.install("ipyvue")
await piplite.install("ipyvuetify")
await piplite.install("ipywidgets")


I had to make a few changes to `glue-jupyter` to make things work in JupyterLite, so we install from our own wheel. In particular, the changes made from regular `glue-jupyter` are:
* I built a wheel for `tornado`, but it didn't seem to load properly in here. So I changed `glue-jupyter` to use the `asyncio` event loop instead (which `tornado.IOLoop` is just a wrapper around anyways).
* The glue-jupyter toolbar uses `traitlets.Any` for `active_tool_id`, which JupyterLite doesn't seem to like. But the tool ID is always just a string or `None`, so we just use `traitlets.Unicode` with `allow_none=True` instead.
* It also seems that JupyterLite doesn't like the `vue_` methods that `ipyvuetify` can expose to Vue from Python - I'm not sure why yet, but I suspect that it's related to how that uses the Jupyter comms. For the histogram viewer widget, I used a hacky tweak where we instead modify a traitlet flag and then call the code that we would've wanted in the observer.

In [ ]:
await piplite.install("pypi/glue_jupyter-0.10.2.dev517+ge72a5fbca-py3-none-any.whl")

In [ ]:
from random import randint
from glue.core import Data, BaseCartesianData
from glue_jupyter import jglue
from glue_jupyter.bqplot.histogram import BqplotHistogramView 
from glue_jupyter.bqplot.scatter import BqplotScatterView

In [ ]:
app = jglue()

In [ ]:
N = 50
x = [randint(0, 50) for _ in range(N)]
y = [randint(0, 50) for _ in range(N)]
z = [randint(0, 50) for _ in range(N)]
data = Data(label="Data", x=x, y=y, z=z)
app.data_collection.append(data)

In [ ]:
from matplotlib import cm

scatter = app.new_data_viewer(BqplotScatterView, data=data)
layer = scatter.layers[0]
data.style.color = "red"
scatter.state.x_att = data.id["x"]
scatter.state.y_att = data.id["y"]
layer.state.size_mode = "Linear"
layer.state.size_att = data.id["z"]
layer.state.cmap_mode = "Linear"
layer.state.fill = True
layer.state.alpha = 0.6
layer.state.size_scaling = 8
layer.state.cmap = cm.viridis

In [ ]:
histogram = app.new_data_viewer(BqplotHistogramView, data=data)
layer.state.x_att = data.id["x"]